# 交叉验证
![cv1](img/cv1.png)

## 让测试数据不参与模型超参数原则
![cv2](img/cv2.png)
![cv3](img/cv3.png)

In [1]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

In [2]:
digits = datasets.load_digits()
X = digits.data
y = digits.target

# train test split

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
        test_size=0.4, random_state = 666)

In [7]:
from sklearn.neighbors import KNeighborsClassifier
best_score, best_p, best_k = 0, 0, 0
for k in range(2, 7):
    for p in range(1, 5):
        knn_clf = KNeighborsClassifier(weights="distance", n_neighbors=k,
                                      p = p)
        knn_clf.fit(X_train, y_train)
        score = knn_clf.score(X_test, y_test)
        if (score > best_score):
            best_score, best_p, best_k = score, p, k
            
print("best k", best_k, " best p:", best_p, " best_score:", best_score)

best k 3  best p: 4  best_score: 0.9860917941585535


## 使用交叉验证

In [8]:
from sklearn.model_selection import cross_val_score

In [9]:
knn_clf = KNeighborsClassifier()
# 分成了3份进行验证
cross_val_score(knn_clf, X_train, y_train)

array([0.98895028, 0.97777778, 0.96629213])

In [10]:
from sklearn.neighbors import KNeighborsClassifier
best_score, best_p, best_k = 0, 0, 0
for k in range(2, 7):
    for p in range(1, 5):
        knn_clf = KNeighborsClassifier(weights="distance", n_neighbors=k,
                                      p = p)
        scores = cross_val_score(knn_clf, X_train, y_train)
        score = np.mean(scores)
        if (score > best_score):
            best_score, best_p, best_k = score, p, k
            
print("best k", best_k, " best p:", best_p, " best_score:", best_score)

best k 2  best p: 2  best_score: 0.9823599874006478


In [11]:
# 用这个模型在测试数据上来看下 到底真正的knn时啥
# 测量最终的准确率到底是多少
best_knn_clf = KNeighborsClassifier(weights="distance", n_neighbors=2, p=2)
best_knn_clf.fit(X_train, y_train)
best_knn_clf.score(X_test, y_test)

0.980528511821975

# 回顾网格搜索

In [17]:
from sklearn.model_selection import GridSearchCV
param_grid = [
    {
        "weights":["distance"],
        "n_neighbors": [i for i in range(2, 7)],
        "p": [i for i in range(1,5)]
    }
]

grid_search = GridSearchCV(KNeighborsClassifier() , param_grid, verbose=1)
grid_search.fit(X_train, y_train)
# 3 folds 意味着将每个训练数据分为了3份数据(2个训练 另外一个验证)
# 20 代表 n_neighbors 有5种选择 p 有4种选择

Fitting 3 folds for each of 20 candidates, totalling 60 fits


[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:   34.2s finished


GridSearchCV(cv=None, error_score='raise',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'weights': ['distance'], 'n_neighbors': [2, 3, 4, 5, 6], 'p': [1, 2, 3, 4]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [18]:
grid_search.best_score_

0.9823747680890538

In [19]:
grid_search.best_params_

{'n_neighbors': 2, 'p': 2, 'weights': 'distance'}

In [20]:
best_knn_clf = grid_search.best_estimator_

In [21]:
best_knn_clf.score(X_test, y_test)

0.980528511821975

In [22]:
# 设置分成的份数 使用cv
# gridsearchcv也可以用
ross_val_score(knn_clf, X_train, y_train, cv=5)

array([0.99543379, 0.96803653, 0.98148148, 0.98130841, 0.98095238])

![kf](img/kfoldcv.png)

LOO-CV: 
    leave one out cv
    有m个数据就有m份
    